In [1]:
from urllib.request import Request
from bs4 import BeautifulSoup
from lyricsgenius import Genius
import pandas as pd
import requests
import json
import os
import re

# store as environment variables
CLIENT_ID = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
CLIENT_SECRET = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
ACCESS_TOKEN = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

def URLWithYear(year):
    URL = f"https://www.officialcharts.com/charts/singles-chart/{year}1231/7501"
    return URL
    
def getArtists(year):
    page = requests.get(URLWithYear(year))
    html = BeautifulSoup(page.text, "html.parser")
    chart_items = html.find_all("div", class_="artist")
    return [item.text.strip().title() for item in chart_items]

def getSongs(year):
    page = requests.get(URLWithYear(year))
    html = BeautifulSoup(page.text, "html.parser")
    chart_items = html.find_all("div", class_="title")
    return [item.text.strip().title() for item in chart_items]



# CREATE DATAFRAME
pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=None)

df = pd.DataFrame()
df["Song"] = getSongs(2021)
df["Artist"] = getArtists(2021)
print(df)

genius = Genius(ACCESS_TOKEN)

lyrics = {}
# currently only working with 2 songs.
# change to 100
def getLyricsFromArtistCharts():
    for i in range(2):
        while True:
            try:
                song_name=df.Song[i]
                artist = genius.search_artist(df.Artist[i], max_songs=0)
                song = genius.search_song(song_name, artist.name)

                lyrics[f"{i}"] = {"artist": artist.name, "song": f"{song_name}", "lyrics": song.lyrics}
                break
            except:
                pass
            
    with open("Lyrics.json", "w") as f:
        json.dump(lyrics, f)


def loadLyrics():     
    with open("Lyrics.json") as f:
        data = json.load(f)
        # print(data)
        return data

                                 Song                    Artist
0                     Merry Christmas   Ed Sheeran & Elton John
1                      Last Christmas                      Wham
2     All I Want For Christmas Is You              Mariah Carey
3               Fairytale Of New York  Pogues Ft Kirsty Maccoll
4   Rockin' Around The Christmas Tree                Brenda Lee
..                                ...                       ...
95                      Little Things                      Abba
96  I Want To Come Home For Christmas             Summer Walker
97                   Jingle Bell Rock              Alessia Cara
98             Kiss Me It'S Christmas      Leona Lewis Ft Ne-Yo
99                           Good 4 U            Olivia Rodrigo

[100 rows x 2 columns]


In [2]:
# getLyricsFromArtistCharts()
# this creates Lyrics.json and has already been run, hence is commented for now.

# load lyrics from json file
data = loadLyrics()
print(data["0"]["lyrics"])

TranslationsEnglishDeutschMerry Christmas Lyrics[Intro: Ed Sheeran]
Build the fire and gather 'round the tree
Fill the glass and maybe come and sing with me

[Chorus: Ed Sheeran & Elton John]
So kiss me under the mistletoe
Pour out the wine, let's toast and pray for December snow
I know there's been pain this year, but it's time to let it go
Next year, you never know
But for now, Merry Christmas, we'll
Dance in the kitchen while embers glow
We've both known love, but this love we got is the bеst of all
I wish you could see you through my eyes, then you would know
My God, you look bеautiful
Right now, Merry Christmas
[Verse 1: Ed Sheeran & Elton John]
The fire is raging on
And we'll all sing along to the song
Just having so much fun
While we're here, can we all
Spare a thought for the ones who have gone?
Merry Christmas, everyone

[Refrain: Choir]
Ah-ah-ah-ah
Ah-ah-ah-ah

[Chorus: Ed Sheeran & Elton John]
So just keep kissin' me under the mistletoe
Pour out the wine, let's toast and pra

In [3]:
from nltk.tokenize import word_tokenize, WhitespaceTokenizer
from nltk.corpus import stopwords

# remove everything between [square brackets]
def removeBrackets(lyrics):
    lyrics = re.sub("\(.*?\)|\[.*?]", "", lyrics)
    return lyrics

#remove new line character
def removeNewLine(lyrics):
    lyrics = re.sub("\\n", " ", lyrics)
    return lyrics

#remove selected words
# Need to remove all words at start beginning with Translations. 
# All subsstrings with 4Embed.
# Decide on other common target words.
# Need to remove punctuation substrings (not apostrophes)
target_words = ["TranslationsEnglishDeutsch", "4Embed", "I'll", "i'll"]
def removeTargetWords(lyrics):
    for i in target_words:
        lyrics = re.sub(f"{i}".lower(), "", lyrics.lower())
    return lyrics

# clean brackets, new line, target words.
def cleanLyrics(lyrics):
    return removeTargetWords(removeNewLine(removeBrackets(lyrics)))

def tokenizeLyrics(lyrics):
    tk = WhitespaceTokenizer()
    return tk.tokenize(lyrics)

def removeStopWords(tokenized_lyrics):
    return [word for word in tokenized_lyrics if not word in stopwords.words()]

def removeUntilLyrics(tokenized_lyrics):
    for i in range(tokenized_lyrics.index("lyrics")):
        tokenized_lyrics.remove(tokenized_lyrics[i])
    return tokenized_lyrics

# remove punctuation on the ends and beginnings of words. But not apostrophes.



"""
might want to keep everything in the dataframe
# but for now since grouping each year's top 100, doesn't matter if all the artist and song data gets mixed together
"""




"\nmight want to keep everything in the dataframe\n# but for now since grouping each year's top 100, doesn't matter if all the artist and song data gets mixed together\n"

In [4]:
# cleans and tokenizes lyrics from json file.
song_lyrics = data["0"]["lyrics"] # only first set of lyrics ["0"]
clean_lyrics = cleanLyrics(song_lyrics).lower()
lyrics_tokenized = tokenizeLyrics(clean_lyrics)
rm_lyrics = removeStopWords(lyrics_tokenized)
processed_lyrics = removeUntilLyrics(rm_lyrics)
# print(processed_lyrics)


song_lyrics2 = data["1"]["lyrics"]
clean_lyrics2 = cleanLyrics(song_lyrics2).lower()
lyrics_tokenized2 = tokenizeLyrics(clean_lyrics2)
rm_lyrics2 = removeStopWords(lyrics_tokenized2)
processed_lyrics2 = removeUntilLyrics(rm_lyrics2)
print(processed_lyrics2)

['traditionallast', 'lyrics', 'ah-ah', 'ooh-woah', 'oh-oh', 'last', 'christmas,', 'gave', 'heart', 'next', 'day,', 'gave', 'away', 'year,', 'save', 'tears', 'give', 'someone', 'special', 'last', 'christmas,', 'gave', 'heart', 'next', 'day,', 'gave', 'away', 'year,', 'save', 'tears', 'give', 'someone', 'special', 'bitten', 'twice', 'shy', 'keep', 'distance,', 'still', 'catch', 'eye', 'tell', 'me,', 'baby,', 'recognise', 'me?', 'well,', 'year,', 'surprise', '"happy', 'christmas,"', 'wrapped', 'sent', 'note', 'saying,', '"i', 'love', 'you,"', 'meant', 'know', 'fool', "i've", 'kissed', 'now,', 'know', 'fool', 'last', 'christmas,', 'gave', 'heart', 'next', 'day,', 'gave', 'away', 'year,', 'save', 'tears', 'give', 'someone', 'special', 'last', 'christmas,', 'gave', 'heart', 'next', 'day,', 'gave', 'away', 'year,', 'save', 'tears', 'give', 'someone', 'special', 'oh,', 'oh,', 'baby', 'crowded', 'room,', 'friends', 'tired', 'eyes', "i'm", 'hiding', 'soul', 'ice', 'god,', 'thought', 'someone', '

In [5]:
# count unique words
total_count = {}
seen = set()

def countUniqueWords(processed_lyrics):
    for word in processed_lyrics:
        if word not in seen:
            seen.add(word)
            total_count[word] = 1
        else:
            total_count[word] += 1
    return total_count

In [6]:
countUniqueWords(processed_lyrics)
countUniqueWords(processed_lyrics2)

{'christmas': 11,
 'build': 1,
 'fire': 4,
 'gather': 1,
 "'round": 1,
 'tree': 1,
 'fill': 1,
 'glass': 1,
 'maybe': 2,
 'sing': 2,
 'kiss': 1,
 'mistletoe': 2,
 'wine,': 2,
 "let's": 2,
 'toast': 2,
 'pray': 2,
 'december': 2,
 'snow': 2,
 'know': 8,
 "there's": 2,
 'pain': 2,
 'year,': 11,
 'time': 7,
 'let': 2,
 'go': 2,
 'next': 9,
 'never': 3,
 'now,': 5,
 'merry': 10,
 'christmas,': 11,
 "we'll": 7,
 'dance': 2,
 'kitchen': 2,
 'embers': 2,
 'glow': 2,
 "we've": 2,
 'known': 2,
 'love,': 3,
 'love': 5,
 'got': 2,
 'bеst': 1,
 'wish': 2,
 'could': 3,
 'see': 2,
 'eyes,': 2,
 'would': 2,
 'god,': 3,
 'look': 2,
 'bеautiful': 1,
 'right': 2,
 'raging': 1,
 'along': 1,
 'song': 1,
 'much': 2,
 'fun': 1,
 "we're": 1,
 'here,': 1,
 'spare': 1,
 'thought': 4,
 'ones': 1,
 'gone?': 1,
 'everyone': 2,
 'ah-ah-ah-ah': 6,
 'keep': 2,
 "kissin'": 1,
 'best': 1,
 'beautiful': 1,
 'feel': 1,
 'comes': 1,
 'every': 1,
 'year': 2,
 "helpin'": 1,
 'us': 1,
 'carry': 1,
 'filled': 1,
 'family': 1